**Análisis Exploratorio de Datos: COVID-19 en Estados Unidos**

In [ ]:
pip install requests pandas typing numpy  scipy matplotlib seaborn seaborn statsmodels scikit-learn


In [25]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os

from src.data_loader import load_all_states_data
from src.data_processor import process_data

# Crear la carpeta para las figuras si no existe
os.makedirs('results/figures', exist_ok=True)

# Configuración de estilo para todas las visualizaciones
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Función para guardar figuras
def save_figure(fig, filename):
    fig.savefig(f"results/figures/{filename}", dpi=300, bbox_inches='tight')
    plt.close(fig)

try:
    # 1️⃣ Carga y limpieza de datos
    print("1. Carga y limpieza de datos")
    raw_data = load_all_states_data()
    data = process_data(raw_data)
    
    # Manejo de valores NaN
    data = data.dropna(subset=['total_cases', 'total_deaths', 'total_tests'])
    data = data.fillna(0)  # Rellena los valores NaN restantes con 0
    
    print("Primeras filas del dataset procesado:")
    print(data.head())
    
    print("\nInformación del dataset:")
    print(data.info())
    
    # 2️⃣ Estadísticas descriptivas
    print("\n2. Estadísticas descriptivas")
    print(data.describe())
    
    # Análisis de valores faltantes
    missing_values = data.isnull().sum()
    print("\nValores faltantes por columna:")
    print(missing_values[missing_values > 0])
    
    # 3️⃣ Visualización de datos
    print("\n3. Visualización de datos")
    
    # Evolución temporal de casos
    print("Generando gráfico de evolución temporal de casos...")
    fig, ax = plt.subplots()
    for state in ['CA', 'NY', 'FL', 'TX']:
        state_data = data[data['state'] == state]
        ax.plot(state_data['date'], state_data['new_cases_7day_avg'], label=state)
    ax.set_title('Promedio Móvil de 7 Días de Nuevos Casos por Estado')
    ax.set_xlabel('Fecha')
    ax.set_ylabel('Nuevos Casos (Promedio 7 Días)')
    ax.legend()
    save_figure(fig, "evolucion_temporal_casos.png")
    
    # Distribución de casos por estado
    print("Generando gráfico de distribución de casos por estado...")
    latest_data = data[data['date'] == data['date'].max()]
    fig, ax = plt.subplots(figsize=(15, 10))
    sns.barplot(x='state', y='cases_per_100k', data=latest_data.sort_values('cases_per_100k', ascending=False), ax=ax)
    ax.set_title('Casos por 100k Habitantes por Estado (Última Fecha)')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    save_figure(fig, "casos_por_estado.png")
    
    # 4️⃣ Análisis de correlaciones
    print("\n4. Análisis de correlaciones")
    numeric_cols = ['total_cases', 'total_deaths', 'cases_per_100k', 'deaths_per_100k', 'positivity_rate', 'case_fatality_rate']
    correlation_matrix = data[numeric_cols].corr()
    
    print("Generando matriz de correlación...")
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', ax=ax)
    ax.set_title('Matriz de Correlación de Métricas COVID-19')
    save_figure(fig, "matriz_correlacion.png")
    
    # 5️⃣ Detección de outliers
    print("\n5. Detección de outliers")
    
    print("Generando gráfico de caja para detección de outliers...")
    fig, ax = plt.subplots(figsize=(12, 6))
    sns.boxplot(x='state', y='new_cases', data=data, ax=ax)
    ax.set_title('Distribución de Nuevos Casos por Estado')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    save_figure(fig, "deteccion_outliers.png")
    
    # Identificación de outliers usando el método IQR
    Q1 = data['new_cases'].quantile(0.25)
    Q3 = data['new_cases'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data['new_cases'] < lower_bound) | (data['new_cases'] > upper_bound)]
    print(f"\nNúmero de outliers detectados: {len(outliers)}")
    print("\nPrimeros 5 outliers detectados:")
    print(outliers[['state', 'date', 'new_cases']].head())

    print("\nAnálisis completado. Las figuras se han guardado en la carpeta 'results/figures/'.")

except Exception as e:
    print(f"Se produjo un error durante el análisis: {str(e)}")


1. Carga y limpieza de datos


c:\Users\Administrator\Desktop\covid19_usa_analysis\notebooks\..\src\data_loader.py:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(all_data, ignore_index=True)


Primeras filas del dataset procesado:
          date state  total_cases  total_deaths  total_tests  new_cases  \
721 2020-03-17    AK          3.0           0.0        337.0        0.0   
720 2020-03-18    AK          8.0           0.0        412.0        5.0   
719 2020-03-19    AK         11.0           0.0        438.0        3.0   
718 2020-03-20    AK         14.0           0.0        698.0        3.0   
717 2020-03-21    AK         15.0           0.0        772.0        1.0   

     new_deaths  new_cases_7day_avg  new_deaths_7day_avg  cases_per_100k  \
721         0.0                 0.0                  0.0        0.410091   
720         0.0                 0.0                  0.0        1.093576   
719         0.0                 0.0                  0.0        1.503667   
718         0.0                 0.0                  0.0        1.913758   
717         0.0                 0.0                  0.0        2.050455   

     deaths_per_100k  positivity_rate  case_fatality_r

c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


Generando gráfico de distribución de casos por estado...


C:\Users\Administrator\AppData\Local\Temp\ipykernel_11140\3562825555.py:77: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=90)



4. Análisis de correlaciones
Generando matriz de correlación...

5. Detección de outliers
Generando gráfico de caja para detección de outliers...


C:\Users\Administrator\AppData\Local\Temp\ipykernel_11140\3562825555.py:98: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=90)



Número de outliers detectados: 1978

Primeros 5 outliers detectados:
    state       date  new_cases
135    AL 2020-10-23     3852.0
95     AL 2020-12-02     3928.0
93     AL 2020-12-04     3840.0
89     AL 2020-12-08     4436.0
87     AL 2020-12-10     4735.0

Análisis completado. Las figuras se han guardado en la carpeta 'results/figures/'.


# Conclusiones
Conclusiones del Análisis Exploratorio de Datos:
1️⃣ Carga y limpieza de datos:

Se encontraron valores NaN en varias columnas, especialmente en las primeras filas del dataset.
Se aplicó una limpieza eliminando filas con valores NaN en columnas críticas (total_cases, total_deaths, total_tests).
Los valores NaN restantes se rellenaron con 0, lo que podría afectar algunos cálculos.
Se observó una advertencia de pandas sobre la concatenación de DataFrames, que se abordó en el código de carga de datos.

2️⃣ Estadísticas descriptivas:

Los datos cubren múltiples estados de EE.UU. con un amplio rango de casos y muertes.
Existe una gran variabilidad en los números de casos y muertes entre estados.
Las tasas de positividad y letalidad también varían significativamente entre estados.
Se observan diferencias notables en el número de pruebas realizadas por estado.

3️⃣ Visualización de datos:

La evolución temporal de casos muestra diferentes patrones entre estados, con algunos experimentando picos más pronunciados.
La distribución de casos por 100,000 habitantes varía considerablemente entre estados, lo que podría indicar diferencias en la propagación o en las estrategias de contención.
Se observan tendencias estacionales en la propagación del virus en algunos estados.

4️⃣ Análisis de correlaciones:

Existe una fuerte correlación positiva entre el total de casos y el total de muertes.
La tasa de positividad muestra una correlación moderada con los casos por 100,000 habitantes, sugiriendo que mayores tasas de positividad se asocian con mayor propagación.
La tasa de letalidad no muestra una fuerte correlación con otras variables, lo que podría indicar que depende de factores no incluidos en este análisis.

5️⃣ Detección de outliers:

Se detectaron varios outliers en el número de nuevos casos diarios.
Estos outliers podrían representar eventos de superpropagación, cambios en los criterios de reporte, o errores en los datos.
Los outliers podrían afectar significativamente los promedios y otras estadísticas, por lo que se debe tener precaución al interpretar los resultados.

Limitaciones del análisis:

Los datos solo llegan hasta 2021, lo que no permite un análisis de la situación más reciente.
No se consideran factores externos como políticas de salud pública, medidas de confinamiento o tasas de vacunación.
La calidad y consistencia de los datos puede variar entre estados debido a diferentes métodos de reporte.
No se han tenido en cuenta factores demográficos más allá de la población total.
El manejo de valores NaN podría afectar la precisión de algunos cálculos.

Recomendaciones para futuros análisis:

Incorporar datos más recientes para tener una visión actualizada de la situación.
Incluir información sobre políticas de salud pública, medidas de confinamiento y tasas de vacunación para entender mejor las diferencias entre estados.
Analizar la relación entre factores demográficos (edad, densidad poblacional, etc.) y la propagación del COVID-19.
Realizar un análisis más detallado a nivel de condado o ciudad para identificar patrones más específicos.
Comparar la situación de EE.UU. con otros países para obtener una perspectiva global.
Implementar técnicas más avanzadas de manejo de datos faltantes y outliers.
Considerar el uso de modelos predictivos para proyectar tendencias futuras basadas en los datos históricos.